<a href="https://colab.research.google.com/github/karaage0703/ai-karaage-sensei-bot/blob/main/notebooks/hatenablog_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はてなブログエクスポートデータのテキスト抽出

Rubyインストール

In [ ]:
!apt -qq install libtool libffi-dev ruby-full make
!apt -qq install libzmq3-dev libczmq-dev
!gem install ffi-rzmq
!gem install specific_install
!gem specific_install https://github.com/SciRuby/iruby
!iruby register

Google Driveとの連携

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Google DriveからGoogle Colabへのデータコピー

In [ ]:
!cp /content/drive/MyDrive/colab_data/karaage.hatenadiary.jp.export.txt ./

変換スクリプトダウンロード

In [ ]:
!wget https://raw.githubusercontent.com/karaage0703/scrapbox_chatgpt_connector/hatenablog/mt2json.rb
!chmod 755 mt2json.rb

Movable Type形式からjson形式に変換

In [ ]:
!ruby ./mt2json.rb karaage.hatenadiary.jp.export.txt > karaage.hatenadiary.jp.json

ファイルをGoogle Driveにコピー

In [ ]:
!cp karaage.hatenadiary.jp.json /content/drive/MyDrive/colab_data/

jsonをテキストに変換

In [ ]:
import json
import re

# Read JSON file
with open("karaage.hatenadiary.jp.json", "r", encoding="utf-8") as json_file:
    json_content = json.load(json_file)

# Function to process and convert JSON content to text
def json_to_flat_txt(json_content):
    txt_content = ""
    for key, value in json_content.items():
        if isinstance(value, list):
            txt_content += f"{key}:\n"
            for item in value:
                txt_content += f"- {item}\n"
        else:
            txt_content += f"{key}: {value}\n"
        txt_content += "\n"

    # Replace \u3000 with a full-width space
    txt_content = txt_content.replace("\\u3000", "　")

    # Remove the specified strings
    remove_strings = ["pages:", "- {'title':", "}", " 'lines':", "'", ",", "[", "]"]
    for string in remove_strings:
        txt_content = txt_content.replace(string, "")

    # Remove HTML tags
    txt_content = re.sub(r'<.*?>', '', txt_content)

    return txt_content

# Convert the JSON content to a flat text format, replace \u3000 with full-width space, remove specific strings, and remove HTML tags
flat_txt_content = json_to_flat_txt(json_content)

# Save the text content to a .txt file
with open("./blog_text.txt", "w", encoding="utf-8") as txt_file:
    txt_file.write(flat_txt_content)

ファイルのGoogle Driveへの移動

In [ ]:
!cp blog_text.txt /content/drive/MyDrive/colab_data/

# 参考リンク
- https://zenn.dev/karaage0703/articles/9bd40f2d3d1a76
- https://zenn.dev/kun432/scraps/1a1c75cd4869ec
- https://gist.github.com/ngoto/61459a11652826416d8f3ff9360f27c2